In [88]:
from enum import IntEnum

class _dataframe_field(IntEnum):
    original_index = 0
    line_flag = 1
    circle_flag = 2
    arc_flag = 3
    point_flag = 4
    start_x = 5
    start_y = 6
    end_x = 7
    end_y = 8
    mid_x = 9
    mid_y = 10
    angle = 11
    length = perimeter = 12
    d_x = 13
    d_y = 14
    u_x = t_x = 15
    u_y = t_y = 16
    n_x = r_x = 17
    n_y = r_y = 18
    center_x = 19
    center_y = 20
    radius = 21
    start_angle = 22
    end_angle = 23
    arc_span = 24
    
    @classmethod
    def count(cls) -> int: return len(cls)

class _edge_attribute(IntEnum):
    parallel = 0
    colinear = 1
    offset = 2
    overlap_ratio = 3
    oblique = 4
    intersection_min = 5
    intersection_max = 6
    angle_difference_sin_min = 7
    angle_difference_cos_min = 8
    angle_difference_sin_max = 9
    angle_difference_cos_max = 10
    
    @classmethod
    def count(cls) -> int: return len(cls)

from typing import Tuple
import torch
from torch import Tensor
from torch import newaxis

In [89]:
import ezdxf

dxf_file = "C:\\Users\\Rafael\\Desktop\\test.dxf"

doc = ezdxf.readfile(dxf_file)
modelSpace = doc.modelspace()

entities = [entity for entity in modelSpace if entity.dxftype() in ('CIRCLE', 'ARC')]

F = _dataframe_field
dataframe = torch.zeros((F.count(), len(entities)), dtype=torch.float32)

for i, entity in enumerate(entities):
    dataframe[F.original_index,i] = i
    entity_type = entity.dxftype()
    
    if entity_type == 'LINE':
        dataframe[F.line_flag,i] = 1
        dataframe[F.start_x,i], dataframe[F.start_y,i], _ = entity.dxf.start
        dataframe[F.end_x,i], dataframe[F.end_y,i], _ = entity.dxf.end
    
    elif entity_type == 'POINT':
        dataframe[F.point_flag,i] = 1
        dataframe[F.mid_x,i], dataframe[F.mid_y,i], _ = entity.dxf.location
        
    elif entity_type in ('CIRCLE', 'ARC'):
        dataframe[F.circle_flag,i] = 1 if entity_type == 'CIRCLE' else 0
        dataframe[F.arc_flag,i] = 1 if entity_type == 'ARC' else 0
        dataframe[F.radius,i] = entity.dxf.radius
        dataframe[F.center_x,i], dataframe[F.center_y,i], _ = entity.dxf.center
        dataframe[F.start_angle,i] = 0 if entity_type == 'CIRCLE' else entity.dxf.start_angle
        dataframe[F.end_angle,i] = 360 if entity_type == 'CIRCLE' else entity.dxf.end_angle
        
# === LINES ===
is_line = dataframe[F.line_flag] == 1

min_length = 1e-3

dataframe[F.mid_x] = torch.where(is_line, (dataframe[F.start_x] + dataframe[F.end_x]) / 2, dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_line, (dataframe[F.start_y] + dataframe[F.end_y]) / 2, dataframe[F.mid_y])

dataframe[F.d_x] = torch.where(is_line, dataframe[F.end_x] - dataframe[F.start_x], dataframe[F.d_x])
dataframe[F.d_y] = torch.where(is_line, dataframe[F.end_y] - dataframe[F.start_y], dataframe[F.d_y])
dataframe[F.angle] = torch.where(is_line, torch.arctan2(dataframe[F.d_y], dataframe[F.d_x]) % (2*torch.pi), dataframe[F.angle])
dataframe[F.length] = torch.where(is_line, torch.sqrt(dataframe[F.d_x]**2 + dataframe[F.d_y]**2), dataframe[F.length])

is_line &= (dataframe[F.length] > min_length)

# Unit direction and unit perpendicular vectors
dataframe[F.u_x] = torch.where(is_line, dataframe[F.d_x] / dataframe[F.length], dataframe[F.u_x])
dataframe[F.u_y] = torch.where(is_line, dataframe[F.d_y] / dataframe[F.length], dataframe[F.u_y])
dataframe[F.n_x] = torch.where(is_line, -dataframe[F.u_y], dataframe[F.n_x])
dataframe[F.n_y] = torch.where(is_line, dataframe[F.u_x], dataframe[F.n_y])

# === POINTS ===
is_point = dataframe[F.point_flag] == 1

for field in [F.start_x, F.end_x]: dataframe[field] = torch.where(is_point, dataframe[F.mid_x], dataframe[field])
for field in [F.start_y, F.end_y]: dataframe[field] = torch.where(is_point, dataframe[F.mid_y], dataframe[field])

is_line_point = (dataframe[F.line_flag] == 1) & (dataframe[F.length] <= min_length)

dataframe[F.point_flag] = torch.where(is_line_point, 1, dataframe[F.point_flag])
dataframe[F.line_flag] = torch.where(is_line_point, 0, dataframe[F.line_flag])

# === CIRCLES AND ARCS ===
is_circle = dataframe[F.circle_flag] == 1
is_arc = dataframe[F.arc_flag] == 1
is_circle_or_arc = is_circle | is_arc

dataframe[F.start_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.start_angle]), dataframe[F.start_angle])
dataframe[F.end_angle] = torch.where(is_circle_or_arc, torch.deg2rad(dataframe[F.end_angle]), dataframe[F.end_angle])

dataframe[F.arc_span] = torch.where(is_circle, 2 * torch.pi, dataframe[F.arc_span])
dataframe[F.arc_span] = torch.where(is_arc, ((dataframe[F.end_angle] - dataframe[F.start_angle]) % (2*torch.pi)), dataframe[F.arc_span])

mid_angle = torch.where(is_circle_or_arc, (dataframe[F.start_angle] + dataframe[F.arc_span] / 2) % (2*torch.pi), 0)

dataframe[F.start_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.start_angle]), dataframe[F.start_x])
dataframe[F.start_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.start_angle]), dataframe[F.start_y])
dataframe[F.end_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(dataframe[F.end_angle]), dataframe[F.end_x])
dataframe[F.end_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(dataframe[F.end_angle]), dataframe[F.end_y])
dataframe[F.mid_x] = torch.where(is_circle_or_arc, dataframe[F.center_x] + dataframe[F.radius] * torch.cos(mid_angle), dataframe[F.mid_x])
dataframe[F.mid_y] = torch.where(is_circle_or_arc, dataframe[F.center_y] + dataframe[F.radius] * torch.sin(mid_angle), dataframe[F.mid_y])

dataframe[F.r_x] = torch.where(is_circle_or_arc, (dataframe[F.mid_x] - dataframe[F.center_x]) / dataframe[F.radius], dataframe[F.r_x])
dataframe[F.r_y] = torch.where(is_circle_or_arc, (dataframe[F.mid_y] - dataframe[F.center_y]) / dataframe[F.radius], dataframe[F.r_y])
dataframe[F.t_x] = torch.where(is_circle_or_arc, -dataframe[F.r_y], dataframe[F.t_x])
dataframe[F.t_y] = torch.where(is_circle_or_arc, dataframe[F.r_x], dataframe[F.t_y])

dataframe[F.perimeter] = torch.where(is_circle_or_arc, dataframe[F.radius] * dataframe[F.arc_span], dataframe[F.perimeter])

In [90]:
# Parameters
line_obb_width=0.5
parallel_max_offset=25.
parallel_angle_tolerance=0.01

obb_width=None; angle_tolerance=None

In [91]:
obb_width = obb_width or line_obb_width
angle_tolerance = (angle_tolerance or parallel_angle_tolerance) * torch.pi/180

elements=dataframe; width=obb_width; length_extension=obb_width

In [92]:
n_elements = elements.size(1)

# Filter supported elements
is_line = elements[F.line_flag] == 1
is_circle_or_arc = (elements[F.circle_flag] == 1) | (elements[F.arc_flag] == 1)

filter = is_line | is_circle_or_arc

obbs = torch.empty((n_elements, 4, 2), dtype=torch.float32)

In [93]:
arcs = elements[:, is_circle_or_arc]

margin = width / 2

r_x, r_y, t_x, t_y = arcs[F.r_x], arcs[F.r_y], arcs[F.t_x], arcs[F.t_y]
arc_span, radius = arcs[F.arc_span], arcs[F.radius]

dx_length = torch.where(arc_span < torch.pi, t_x * (radius * torch.sin(arc_span/2) + margin), t_x * (radius + margin))
dy_length = torch.where(arc_span < torch.pi, t_y * (radius * torch.sin(arc_span/2) + margin), t_y * (radius + margin))

dx_width = r_x * (radius * (1 - torch.cos(arc_span/2)) + margin)
dy_width = r_y * (radius * (1 - torch.cos(arc_span/2)) + margin)

dx_margin = r_x * margin
dy_margin = r_y * margin

corner1 = torch.stack([arcs[F.mid_x] - dx_length + dx_margin,
                        arcs[F.mid_y] - dy_length + dy_margin], dim=1)

corner2 = torch.stack([arcs[F.mid_x] + dx_length + dx_margin,
                        arcs[F.mid_y] + dy_length + dy_margin], dim=1)

corner3 = torch.stack([arcs[F.mid_x] + dx_length - dx_width,
                        arcs[F.mid_y] + dy_length - dy_width], dim=1)

corner4 = torch.stack([arcs[F.mid_x] - dx_length - dx_width,
                        arcs[F.mid_y] - dy_length - dy_width], dim=1)

obbs[is_circle_or_arc] = torch.stack([corner1, corner2, corner3, corner4], dim=1)

In [95]:
obbs[is_circle_or_arc] == obbs_before

tensor([[[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True

In [87]:
obbs_before

tensor([[[ 9.7500e+00,  4.0250e+01],
         [ 9.7500e+00, -4.0250e+01],
         [ 9.0250e+01, -4.0250e+01],
         [ 9.0250e+01,  4.0250e+01]],

        [[ 1.3958e+02,  4.7696e+01],
         [ 1.0174e+02,  7.4108e+00],
         [ 1.1001e+02, -3.5338e-01],
         [ 1.4784e+02,  3.9932e+01]],

        [[ 2.0975e+02,  4.0250e+01],
         [ 2.0975e+02, -4.0250e+01],
         [ 2.9025e+02, -4.0250e+01],
         [ 2.9025e+02,  4.0250e+01]],

        [[ 3.4143e+02,  5.3377e+01],
         [ 2.9601e+02, -2.7166e+00],
         [ 3.1368e+02, -1.7026e+01],
         [ 3.5910e+02,  3.9068e+01]],

        [[ 4.0975e+02,  4.0250e+01],
         [ 4.0975e+02, -4.0250e+01],
         [ 4.9025e+02, -4.0250e+01],
         [ 4.9025e+02,  4.0250e+01]],

        [[ 5.0975e+02,  4.0250e+01],
         [ 5.0975e+02, -4.0250e+01],
         [ 5.9025e+02, -4.0250e+01],
         [ 5.9025e+02,  4.0250e+01]],

        [[ 6.0975e+02,  4.0250e+01],
         [ 6.0975e+02, -4.0250e+01],
         [ 6.6697e+02, -4.

In [73]:
obbs_before = obbs[is_circle_or_arc]
obbs_before

tensor([[[ 9.7500e+00,  4.0250e+01],
         [ 9.7500e+00, -4.0250e+01],
         [ 9.0250e+01, -4.0250e+01],
         [ 9.0250e+01,  4.0250e+01]],

        [[ 1.3958e+02,  4.7696e+01],
         [ 1.0174e+02,  7.4108e+00],
         [ 1.1001e+02, -3.5338e-01],
         [ 1.4784e+02,  3.9932e+01]],

        [[ 2.0975e+02,  4.0250e+01],
         [ 2.0975e+02, -4.0250e+01],
         [ 2.9025e+02, -4.0250e+01],
         [ 2.9025e+02,  4.0250e+01]],

        [[ 3.4143e+02,  5.3377e+01],
         [ 2.9601e+02, -2.7166e+00],
         [ 3.1368e+02, -1.7026e+01],
         [ 3.5910e+02,  3.9068e+01]],

        [[ 4.0975e+02,  4.0250e+01],
         [ 4.0975e+02, -4.0250e+01],
         [ 4.9025e+02, -4.0250e+01],
         [ 4.9025e+02,  4.0250e+01]],

        [[ 5.0975e+02,  4.0250e+01],
         [ 5.0975e+02, -4.0250e+01],
         [ 5.9025e+02, -4.0250e+01],
         [ 5.9025e+02,  4.0250e+01]],

        [[ 6.0975e+02,  4.0250e+01],
         [ 6.0975e+02, -4.0250e+01],
         [ 6.6697e+02, -4.

In [10]:
torch.tensor([[0, 1, 2, 3], [4, 5, 6, 7]]) == torch.tensor([[0, 1, 2, 3], [4, 5, 6, 7]])

tensor([[True, True, True, True],
        [True, True, True, True]])